<a href="https://colab.research.google.com/github/saiashirwad/relation-prediction-2/blob/master/Thesis_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
cd /content/drive/My Drive/Thesis/code/relation-prediction-2

/content/drive/My Drive/Thesis/code/relation-prediction-2


In [0]:
pip install torchkge --quiet

In [0]:
pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html --quiet

In [0]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
import importlib

In [0]:
from tqdm import *
import tqdm

In [0]:
from layers import SpecialSpmmFinal

In [0]:
import train 
importlib.reload(train)
from train import *

In [0]:
import torch
import torch.nn as nn 

from functools import reduce
from operator import mul

from torch_scatter import scatter
from embedding import EmbeddingMul2

In [0]:
kg_train, kg_test, kg_val = load_fb15k237()
args = Args(100, 200, 100, 2, 100, 2000, 0.001, 10, 'cuda', 'sgd')

In [0]:
n_ent, n_rel = kg_train.n_ent, kg_train.n_rel
total_triplets = get_valid_triplets(kg_train, kg_test, kg_val)

Number of unique triplets: 620232


In [0]:
dataloader = DataLoader(kg_train, batch_size=args.batch_size, shuffle=False, pin_memory=cuda.is_available())
ent_embed, rel_embed = get_init_embed()

In [0]:
class KGLayer(nn.Module):
    def __init__(self, n_entities, n_relations, ent_embed, rel_embed, in_dim, out_dim, concat=True, device="gpu"):
        super(KGLayer, self).__init__()

        self.n_entities = n_entities
        self.n_relations = n_relations
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.device = device

        self.a = nn.Linear(3 * in_dim, out_dim).to(device)
        nn.init.xavier_normal_(self.a.weight.data, gain=1.414)

        self.concat = concat

        self.a_2 = nn.Linear(out_dim, 1).to(device)
        nn.init.xavier_normal_(self.a_2.weight.data, gain=1.414)

        self.special_spmm_final = SpecialSpmmFinal()

        # self.ent_embed = EmbeddingMul2(n_entities, in_dim, ent_embed, True, torch.device(device))
        # self.rel_embed = EmbeddingMul2(n_relations, in_dim, rel_embed, True, torch.device(device))

        self.ent_embed = nn.Embedding(n_entities, in_dim).to(device)
        self.rel_embed = nn.Embedding(n_relations, in_dim).to(device)
    
    def forward(self, triplets):
        N = self.n_entities

        h = torch.cat((
            self.ent_embed(triplets[:, 0]),
            self.ent_embed(triplets[:, 1]),
            self.rel_embed(triplets[:, 2])
        ), dim=1)
        c = self.a(h)
        b = F.leaky_relu(self.a_2(c))
        e_b = torch.exp(b)

        temp = triplets.t()
        edges = torch.stack([temp[0], temp[1]])

        e_b_sum = self.special_spmm_final(edges, e_b, N, e_b.shape[0], 1)
        temp1 = e_b * c

        h_sum = self.special_spmm_final(edges, temp1,  N, e_b.shape[0], self.out_dim)

        hs = h_sum
        ebs = e_b_sum
        ebs[ebs == 0] = 1e-12


        h_ent = hs / ebs

        index = triplets[:, 2]
        h_rel = scatter(temp1, index=index, dim=0, reduce="mean")

        # del h 
        # torch.cuda.empty_cache()

        if self.concat:
            return F.elu(h_ent), F.elu(h_rel)
        else:
            return h_ent, h_rel

In [0]:
model = KGLayer(n_ent, n_rel, ent_embed, rel_embed, 100, 100, True, "cuda")

In [0]:
optimizer = SGD(model.parameters(), lr=args.lr)

In [0]:
batches = [b for b in dataloader]

In [0]:
for epoch in tnrange(10):
    losses = []
    for i in tnrange(len(batches)):
        batch = batches[i]
        triplets = torch.stack(batch)
        triplets, labels, nodes, edges = negative_sampling(triplets, n_ent, args.negative_rate)
        triplets, labels = triplets.to(args.device), labels.to(args.device)
    
        model.zero_grad()
    
        # start = time.time()
        model.train()
        ent_embed_, rel_embed_ = model(triplets)
        loss = loss_func2(triplets, args.negative_rate, ent_embed_, rel_embed_, device="cuda")
        loss.backward()
        optimizer.step()
    
        # del triplets
        # del labels
        # del ent_embed_
        # del rel_embed_
        # del nodes 
        # del edges  
        torch.cuda.empty_cache()
    
        losses.append(loss.item())
        # print(loss.item())
        # del loss 
        torch.cuda.empty_cache()
    
    print(sum(losses) / len(losses))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """Entry point for launching an IPython kernel.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  This is separate from the ipykernel package so we can avoid doing imports until



4.792156094182147



4.764670699182218



4.735600196532089


...


4.6560647818293885



4.62794095060251



4.605342777976154



4.584204036824024



4.566668677504045



In [0]:
model.eval()
validate(model, kg_val, total_triplets, 100, 'cuda')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



Head Rank Mean : 10996.755 | Hits@10 : 0
Tail Rank Mean : 11081.63 | Hits@10 : 0
Filtered Head Rank Mean: 10996.755
Filtered Tail Rank MEan: 11081.63



In [0]:
from typing import Set, Tuple
def validate(model: KGLayer, kg: KnowledgeGraph, total_triplets: Set[Tuple], batch_size=1000, device='cuda'):
    batch_size = 1


    n_ranks = 10
    k = kg.n_ent

    dataloader = DataLoader(kg, batch_size, shuffle=True)

    hits = [[] for _ in range(10)]
    ranks = [] 

    rankHs, rankTs = [], []

    head_rank_mean, tail_rank_mean, filtered_head_rank_mean, filtered_tail_rank_mean = [0] * 4

    head_hits_10_raw, head_hits_10_filter, tail_hits_10_raw, tail_hits_10_filter = [0] * 4

    data = [d for d in dataloader]

    for i in tnrange(300):
        batch = data[i]
        
        src, dst, rel = batch

        if device == 'cuda':
            src = src.to(device)
            dst = dst.to(device)
            rel = rel.to(device)

        src = model.ent_embed(src)
        dst = model.ent_embed(dst)
        rel = model.ent_embed(rel)

        loss = torch.norm(src + rel - dst, 2, 1)
        loss = loss.repeat(kg.n_ent)


        src_ = src.repeat(kg.n_ent, 1)
        dst_ = dst.repeat(kg.n_ent, 1)
        rel_ = rel.repeat(kg.n_ent, 1)

        dist_head_prediction = model.ent_embed.weight + rel_ - dst_
        dist_tail_prediction = src_ + rel_ - model.ent_embed.weight

        _, head_prediction = torch.topk(torch.sum(torch.abs(dist_head_prediction), dim=1), k=k)
        _, tail_prediction = torch.topk(torch.sum(torch.abs(dist_tail_prediction), dim=1), k=k)

        head_prediction = head_prediction.cpu()
        tail_prediction = tail_prediction.cpu()

        head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter = [0] * 4


        s, d, r = [b[0] for b in batch] # Ugh

        for candidate in head_prediction:
            if candidate == s:
                break
            else:
                head_rank_raw += 1 
                if (candidate, d, r) in total_triplets:
                    continue
                else:
                    head_rank_filter += 1
        
        for candidate in tail_prediction:
            if candidate == d:
                break 
            else:
                tail_rank_raw += 1
                if (s, candidate, r) in total_triplets:
                    continue
                else:
                    tail_rank_filter += 1 

        head_rank_mean += head_rank_raw
        tail_rank_mean += tail_rank_raw

        filtered_head_rank_mean += head_rank_filter
        filtered_tail_rank_mean += tail_rank_filter

        if head_rank_raw < 10:
            head_hits_10_raw += 1
        
        if tail_rank_raw < 10:
            tail_hits_10_raw += 1

        if head_rank_filter < 10:
            head_hits_10_filter += 1
        
        if tail_rank_filter < 10:
            tail_hits_10_filter += 1
        
    
    head_rank_mean /= 200
    tail_rank_mean /= 200

    filtered_head_rank_mean /= 200
    filtered_tail_rank_mean /= 200

    print(f'Head Rank Mean : {head_rank_mean} | Hits@10 : {head_hits_10_raw}')
    print(f'Tail Rank Mean : {tail_rank_mean} | Hits@10 : {tail_hits_10_raw}')

    print(f'Filtered Head Rank Mean: {filtered_head_rank_mean}')
    print(f'Filtered Tail Rank MEan: {filtered_tail_rank_mean}')


    print()


In [0]:
model.eval()
validate(model, kg_val, total_triplets, 100, 'cuda')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



Head Rank Mean : 10868.355 | Hits@10 : 1
Tail Rank Mean : 10559.295 | Hits@10 : 0
Filtered Head Rank Mean: 10868.355
Filtered Tail Rank MEan: 10559.295



In [0]:
!nvidia-smi

Wed Apr  8 15:35:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    31W / 250W |    875MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
dataloader = DataLoader(kg_val, 1, shuffle=True)
data = [d for d  in dataloader]
d = data[0]
model.eval()

KGLayer(
  (a): Linear(in_features=300, out_features=100, bias=True)
  (a_2): Linear(in_features=100, out_features=1, bias=True)
  (special_spmm_final): SpecialSpmmFinal()
  (ent_embed): Embedding(14541, 100)
  (rel_embed): Embedding(237, 100)
)

In [0]:
head = model.ent_embed(d[0].to("cuda"))
tail = model.ent_embed(d[1].to("cuda"))
rel  = model.rel_embed(d[2].to("cuda"))

In [0]:
target_loss = torch.norm(head + rel - tail, 2).repeat(n_ent, 1)

In [0]:
tmphead = head.repeat(n_ent, 1)
tmptail = tail.repeat(n_ent, 1)
tmprel = rel.repeat(n_ent, 1)

In [0]:
tmpHloss = torch.norm(model.ent_embed.weight + tmprel - tmptail, 2, 1).view(-1, 1)

In [0]:
tmpHloss

tensor([[15.2286],
        [13.6497],
        [17.3007],
        ...,
        [15.7120],
        [15.6428],
        [14.6578]], device='cuda:0', grad_fn=<ViewBackward>)

In [0]:
tmpTloss = torch.norm(tmphead + rel - model.ent_embed.weight, 2, 1).view(-1, 1)

In [0]:
tmpTloss

tensor([[16.9137],
        [16.8444],
        [17.3527],
        ...,
        [16.3008],
        [16.8269],
        [17.7307]], device='cuda:0', grad_fn=<ViewBackward>)

In [0]:
rankH = torch.nonzero(F.relu(target_loss - tmpHloss)).size()[0]

In [0]:
rankH

11215

In [0]:
rankT = torch.nonzero(F.relu(target_loss - tmpTloss)).size()[0]

In [0]:
rankT

2449

In [0]:
def fast_validate(model, kg, total_triplets, device):
    dataloader = DataLoader(kg, 1, shuffle=True)
    data = [d for d in dataloader]

    ranks = []

    for i in tnrange(len(data)):
        triplet = data[i]
        if i:
            head = model.ent_embed(triplet[0].to("cuda"))
            tail = model.ent_embed(triplet[1].to("cuda"))
            rel  = model.rel_embed(triplet[2].to("cuda"))
    
            targetLoss = torch.norm(head + rel - tail, 2).repeat(kg.n_ent, 1)
    
            tmpHead = head.repeat(kg.n_ent, 1)
            tmpTail = tail.repeat(kg.n_ent, 1)
            tmpRel  = rel.repeat(kg.n_ent, 1)
    
            tmpHloss = torch.norm(model.ent_embed.weight + tmpRel - tmpTail, 2, 1).view(-1, 1)
            tmpTloss = torch.norm(tmpHead + tmpRel - model.ent_embed.weight, 2, 1).view(-1, 1)
    
            rankH = torch.nonzero(F.relu(targetLoss - tmpHloss)).size()[0]
            rankT = torch.nonzero(F.relu(targetLoss - tmpTloss)).size()[0]
    
            ranks.append( (rankH + rankT + 2) / 2 )
    
    return sum(ranks) / len(ranks)

In [0]:
fast_validate(model, kg_val, total_triplets, "cuda")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  import sys


7147.21172733936

In [0]:
from tqdm import tnrange, tqdm_notebook
from time import sleep

for i in tnrange(4, desc='1st_loop'):
    for j in tnrange(100, desc="2nd loop"):
        sleep(0.01)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """


In [0]:
for i in tnrange(len(data)):
    sleep(0.1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """Entry point for launching an IPython kernel.


KeyboardInterrupt: ignored

In [0]:
def eval_final()